## Geyser Eruptions and Fisher Information

### Carolyn P. Johnston, March 6, 2016

Over Christmas vacation, Husband and Grown Children went off for a drive somewhere, and when they came back, they were arguing about how to solve the following problem: 

*There are 3 geysers: A, B, and C. Geyser A erupts every 2 hours; B, every 4 hours; and C, every 6 hours. The geysers erupt independently (as far as we know).*  

*You arrive at the geysers, with no knowledge of when any of them last erupted. What is the probability that A will erupt before both B and C?*

In the interest of family harmony at Christmas, I got right to work settling the issue. I worked out a closed-form solution, and then used a Monte Carlo simulation to check my work. After several tries at calculating the integral and getting results that didn't match the simulation's (the integral was different every time I worked it out -- not a good sign), I finally calculated the integral correctly (I'd like to be able to blame Probability or at least Calculus, but Arithmetic was actually at fault).


### The analytic solution

The time until next eruption of A, $t_A$, is modeled with $U_A$, a uniform distribution over $[0,2]$; $t_B$, as $U_B$ = Uniform $[0,4]$; $t_C$, as $U_C$ = Uniform$[0,6]$. The events of A, B, and C erupting are assumed to be independent. 

Since the eruption events are independent, the joint probability is given by the product of the individual pdfs:

![alt text](Geyser-eq1.png "Equation 1")

<!-- (originally) $$P(t_A < a, t_B < b, t_C < c) = \int_0^a \int_0^b\int_0^c U_A(a)U_B(b)U_C(c) \cdot da \cdot db \cdot dc.$$ -->

The probability we are interested in is that of the event that $t_A < t_B$ and $t_A < t_C$. This is given by the integral: 

![alt text](Geyser-eq2.png "Equation 2")

<!-- (originally)  $$\int_0^2 \int_a^4\int_a^6 U_A(a)\cdot U_B(b)\cdot U_C(c) \cdot da \cdot db \cdot dc = $$ -->

<!-- (originally)  $$\frac{1}{48} \int_0^2 \left\{\int_a^4 db \cdot \int_a^6 dc\right\} \cdot da = \frac{1}{48} \int_0^2 (4-a)\cdot(6-a) \cdot da = \frac{1}{48} \cdot [\frac{56}{3} + 12] = 0.63888.....$$ -->



### The simulation in R

Again, the time until next eruption of A, $t_A$, is a uniform distribution over $[0,2]$; $t_B$ is Uniform $[0,4]$; $t_C$ is Uniform$[0,6]$, and the $t_i$ for $i=A,B,C$ are assumed to be independent.

I simulated $N$ draws from these distributions, and look at the frequency of the event of interest (A erupting before both B and C, or $t_A < t_B$ and $t_A < t_C$).

I am a little disappointed that with $N=100000$ draws, I only get these estimated probability to the order of a thousandth.

In [1]:

N=100000

getFreq <- function(N)
    {
    # simulate N independent observations of the time till each geyser erupts
    time_till_A_erupts = runif(N, min=0, max=2)
    time_till_B_erupts = runif(N, min=0, max=4)
    time_till_C_erupts = runif(N, min=0, max=6)

    # calculate the proportion of times that the desired event occurs
    events = (time_till_A_erupts < time_till_B_erupts & time_till_A_erupts < time_till_C_erupts)
    frequency = mean(events)
    }

freq1 = getFreq(N)
sprintf("The frequency of A erupting before B and C is estimated at: %f with: %d samples", freq1, N )
sprintf("The difference between this value and the analytic value is: %f", (56.0/3.0 + 12.0)/48.0 - freq1)

[1] "The frequency of A erupting before B and C is estimated at: 0.637260 with: 100000 samples"

[1] "The difference between this value and the analytic value is: 0.001629"

### Exploring the accuracy of my estimates

Why doesn't this estimate converge faster? 
 
Just to be sure I'm not crazy, I drew up a table showing the variance of the error in my estimates as a function of N. It shows that the variance of the error in my estimates drops at a rate of $1/N$, where the multiplier is in the range 0.1-0.3 or so. 

In [2]:
replicate.K.times <- function(N,K=50)
{
    #cat(sprintf("Inside function, N=%d\n", N))
    args = rep(N, K)
    truefreq = (56.0/3.0 + 12.0)/48.0
    truths = rep(truefreq, K)

    # get K estimates of the probability
    freqs = sapply(args, getFreq) 

    # calculate the errors in the estimates
    errors = freqs-truths
    
    # the theoretical distribution of errors is mean 0, since the mean (proportion) estimator is unbiased
    # find the standard deviation of the errors as an estimate of the size of the error
    v_errors = var(errors)
    return(v_errors)
}

N.values = c(100,1000, 10000,100000)
error_sizes = sapply(N.values,replicate.K.times)
data = data.frame(N=N.values, error_sizes=error_sizes)
data

,N,error_sizes
1,100,0.002405878
2,1000,0.0002689388
3,10000,2.40731e-05
4,1e+05,2.859332e-06


This happens because the analytic integral is estimated, in my simulation, by taking the mean of random, independent draws from a Bernoulli r.v. with parameter $\phi\approx$ 0.63888, and therefore the estimate is also an unbiased estimator of $\phi$. 

The Cramer-Rao bound theorem says that its (population) variance, as a function of N, is bounded below by the absolute value of the inverse of the Fisher information at $\phi$; in this case,

![alt text](Geyser-eq3.png "Equation 3")

<!-- (originally) $$\text{lower bound }(\phi) = \frac{1}{I(\phi)} = \frac{\phi(1 - \phi)}{N} \approx\frac{.23}{N}.$$ -->

Therefore the apparent variance of my (unbiased) estimate is close to the minimum variance  estimate for this quantity (in fact, the population variance of this estimator does achieve that lower bound). Unless I want to switch to using a biased estimator, I can't do any better on average than this. 